# 7. Etude du jeu de données California

Ce dataset contient la valeur des logements en Californie par district.

Nous cherchons à prédire la variable `median_house_value`, cette variable étant numérique et continue, nous avons à faire à un problème de régression.

In [ ]:
import numpy as np
import pandas as pd
import math, pprint
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

## 7.1 Chargement du jeu de données

In [ ]:
df = pd.read_csv('/data/housing.csv')

## 7.2 Analyse exploratoire des données

### 7.2.1 Statistiques

Dimensions de notre dataset :

In [ ]:
df.shape

Extrait :

In [ ]:
df.sample(10)

Types des variables :

In [ ]:
df.dtypes

Description des variables numériques :

In [ ]:
df.describe()

Description des variables catégorielles :

In [ ]:
for col in df.select_dtypes(include=[object]).columns:
    counts = df[col].value_counts()
    if len(counts) < 20:
        print("\nModalités de la variable ", col)
        pprint.pprint(df[col].value_counts())
    else:
        print("\nLa variable %s possède %i modalités" % (col, len(counts)))

Analyse des corrélations :

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(df.corr())
plt.show()

Valeurs manquantes :

In [ ]:
pd.isnull(df).sum()

In [ ]:
df.select_dtypes(include=[np.number])

Boîtes à moustaches - version sns :

In [ ]:
def boxplot_grid_sns(df, nb_grid_cols=3, figsize=(16,15)):
    df_num = df.select_dtypes(include=[np.number])
    nb_num_vars = len(df_num.columns)
    fig, axes = plt.subplots(math.ceil(nb_num_vars/nb_grid_cols), nb_grid_cols, figsize=figsize)
    for i, col in enumerate(df_num.columns):
        sns.boxplot(data=df_num, x=col, ax=axes[int(i/nb_grid_cols),i%nb_grid_cols])
    plt.show()
boxplot_grid_sns(df)

Boîtes à moustaches - version plotly :

In [ ]:
def boxplot_grid_plotly(df, nb_grid_cols=3):
    cols = df.select_dtypes(include=[np.number]).columns
    fig = make_subplots(rows=math.ceil(len(cols)/nb_grid_cols), cols=nb_grid_cols)
    for i, var in enumerate(cols):
        fig.add_trace(
            go.Box(y=df[var],
            name=var),
            row=int(i/nb_grid_cols)+1, col=i%nb_grid_cols+1
        )
    fig.update_layout(
        autosize=False,
        width=950,
        height=800,
        showlegend=False
    )
    fig.show()
boxplot_grid_plotly(df)

Histogrammes avec estimation de densité :

In [ ]:
def density_plot_grid(df, nb_grid_cols=3, figsize=(16,15)):
    df_num = df.select_dtypes(include=[np.number])
    nb_num_vars = len(df_num.columns)
    fig, axes = plt.subplots(math.ceil(nb_num_vars/nb_grid_cols), nb_grid_cols, figsize=figsize)
    for i, col in enumerate(df_num.columns):
        not_null_col = df_num[col][df_num[col].notnull()]
        sns.histplot(not_null_col, ax=axes[int(i/nb_grid_cols),i%nb_grid_cols], kde=True)
    plt.show()
density_plot_grid(df)

- Il y a des valeurs manquantes pour la variable `total_bedrooms`. Nous traiterons les valeurs manquantes dans la chaîne de transformation pour le pré-traitement des données.
- Les valeurs de la cible `median_house_value` supérieures à `500000` semblent avoir subi un effet de seuil et avoir été enregistrées avec un montant associé fixé à `500000`. Il semble préférable de ne pas conserver ces enregistrements. Par ailleurs, observons qu'ils se concentrent sur des zones bien spécifiques de la Californie.
- Les variables `total_rooms`, `total_bedrooms`, `population` et `households` sont très fortement corrélées. Il semble pertinent de rapporter le nombre de pièces et la population au nombre de foyers. De même, le nombre de chambres peut être ramené au nombre de pièces. Nous proposons d'introduire les variables `rooms_per_household`, `population_per_household` et `bedrooms_per_room`.
- Certaines boîtes à moustaches indiqueraient un grand nombre de valeurs aberrantes. Cependant, les histogrammes avec estimateur de densité montrent que les distributions correspondantes ne sont pas normales mais asymétriques avec plus d'observations du côté des valeurs élevées.
- Une standardisation ou normalisation sont à tester, en particulier dans le cas de modèles linéaires, car les échelles des variables sont très différentes.

## 7.2.2 Étude des prix de maisons médians supérieurs à 500 000

Commencons par afficher le nombre de lignes pour lesquels la `median_house_value` a été seuillée à 500k :

In [ ]:
(df['median_house_value']>500000).sum()

Ajoutons une colonne indiquant si les lignes ont un prix seuillé :

In [ ]:
df['500k'] = df['median_house_value']>500000
df['500k'] = df['500k'].astype(int);

Puis affichons les données sur une carte avec une couleur différente pour les districts qui ont vu leur `median_house_value` seuillée :

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.3,
        s=df["population"]/100, label="population",
        figsize=(10,8), c="500k", cmap=plt.get_cmap("rainbow"))
plt.legend();

Ces districts semblent être localisés autour des villes de Los Angeles et San Fransisco, ces lignes seront potentiellement génantes pour apprendre un regresseur, aussi nous les supprimons (et nous supprimons la colonne `500k` qui ne sera plus utile par la suite) :

In [ ]:
df = df.drop(columns=['500k']);
df = df.drop(df[df.median_house_value > 500000].index)
len(df)

## 7.2.3 Étude des valeurs manquantes

Une stratégie possible pour la gestion des valeurs manquantes serait de supprimer les lignes correspondantes. Voir ci-dessous une manière de le faire. Cependant, nous préférons gérer les valeurs manquantes au sein de la chaîne de transformation qui opérera l'ensemble des pré-traitements. Ainsi, nous pouvons comparer par validation croisée différentes formes d'imputation pour la gestion des valeurs manquantes. Cette approche est également plus robuste si, pour la mise en production du modèle, d'autres variables que `total_bedrooms` ont des valeurs manquantes.

In [ ]:
#df = df[df['total_bedrooms'].notnull()].copy()

### 7.2.4 Transformation des variables fortement corrélées

Nous calculons les nouvelles variables `rooms_per_household`, `population_per_household` et `bedrooms_per_room`. Nous retirons les variables `total_rooms`, `total_bedrooms` et `population`.

In [ ]:
df['rooms_per_household'] = df['total_rooms'] / df['households']
df['bedrooms_per_room'] = df['total_bedrooms'] / df['total_rooms']
df['population_per_household'] = df['population'] / df['households']
df = df.drop(columns=['total_rooms', 'total_bedrooms', 'population'])

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(df.corr())
plt.show()

Nous observons que le revenu médian est corrélé positivement avec le nombre de pièces mais négativement avec la proportion de chambres à coucher.

### 7.2.5 Visualisations géographiques

Nous pouvons afficher les individus de notre dataset sur un plan dont les axes sont la longitude et la latitude :

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1);

Cette représentation peut-être améliorée en changeant la couleur des points en fonction de la `median_house_value` et la taille en fonction de la `population_per_household` :

In [ ]:
df.plot(kind="scatter", x="longitude", y="latitude", alpha=0.3,
        s=df["population_per_household"], label="population per household",
        figsize=(10,8), c="median_house_value",
        cmap=plt.get_cmap("rainbow"), colorbar=True)
plt.legend();

Le graphique précédent nous permet de constater qu'il y a probablement une erreur sur certains districts car ils font apparaitre des `population_per_household` très grand. Regardons cela plus en détail :

In [ ]:
df[df.population_per_household > 20]

Ces lignes seront problématique pour notre regresseur, aussi nous choisissons de les supprimer en choisissant un seuil au dela duquel les valeurs nous semblent déraisonnables :

In [ ]:
df = df.drop(df[df.population_per_household > 20].index)

## 7.3 Construction du jeu d'entraînement et du jeu de test

Après cette étape d'analyse exploratoire, de transformation de variables et de nettoyage de nos données, nous pouvons maintenant séparer notre dataset en une partie d'entrainement et une partie de test :

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=77)

X_train = train.drop("median_house_value", axis=1)
y_train = train["median_house_value"].copy()
X_test = test.drop("median_house_value", axis=1)
y_test = test["median_house_value"].copy()

## 7.4 Transformation des variables

### 7.4.1 Classement des variables en fonction de leurs types (numériques, catégorielles, ordinales...)

In [ ]:
list(X_train)

In [ ]:
var_names_num = ['longitude', 'latitude', 'housing_median_age', 
                 'households', 'median_income', 
                 'rooms_per_household', 'bedrooms_per_room', 'population_per_household' ]
var_names_cat = ['ocean_proximity']

### 7.4.2 Création des chaînes de transformation pour la préparation des données

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler())
])

preparation_pipeline = ColumnTransformer([
    ("num", num_pipeline, var_names_num),
    ("cat", OneHotEncoder(), var_names_cat),
])

## 7.5 Exploration des modèles

### 7.5.1 Modèle Ridge

#### Choix des hyper-paramètres par validation croisée

In [ ]:
from sklearn.linear_model import Ridge

ridge_pipeline = Pipeline([
    ("preparation", preparation_pipeline),
    ("model", Ridge(tol=0.01, solver="saga"))
])

ridge_param_grid = [
    {
        'model__alpha': np.logspace(-5, 5, 10),
        'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent']
    }
]

gridsearch_ridge = GridSearchCV(
    ridge_pipeline, ridge_param_grid, cv=3, scoring="neg_mean_squared_error", n_jobs=-1
)
gridsearch_ridge.fit(X_train, y_train)

print('Meilleurs paramètres :')
print(gridsearch_ridge.best_params_)

rmse_best_ridge_model = np.sqrt(-gridsearch_ridge.best_score_)
print('RMSE par validation croisée du meilleur modèle : ', rmse_best_ridge_model)

best_ridge_model = gridsearch_ridge.best_estimator_

#### Sauvegarde du meilleur modèle

La cellule suivante sauvegarde le modèle (i.e. : tout le pipeline) dans un fichier qui pourra être réutilisé ultiérieurement (voir à la fin du notebook pour un exemple d'utilisation).

In [ ]:
from joblib import dump, load

dump(best_ridge_model, 'california_ridge_model.joblib')

#### Score par validation croisée sur le jeu d'entraînement

In [ ]:
ridge_cv_scores = cross_val_score(
    best_ridge_model, X_train, y_train, scoring="neg_mean_squared_error", cv=10, n_jobs=-1
)
ridge_cv_scores = np.sqrt(-ridge_cv_scores)
print(
    "Moyenne des scores : ", ridge_cv_scores.mean(),
    "Ecart type des scores : ", ridge_cv_scores.std()
)
dump(ridge_cv_scores, 'california_ridge_score.joblib')

### 7.5.2 Modèle de forêt aléatoire

#### Choix des hyper-paramètres par validation croisée

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf_pipeline = Pipeline([
    ("preparation", preparation_pipeline),
    ("model", RandomForestRegressor(n_jobs=-1, n_estimators=50, random_state=77, criterion='mse'))
])

rf_param_grid = [
    {
        'preparation__num__imputer__strategy': ['mean', 'median', 'most_frequent'],
        'model__min_samples_leaf': [1,3,5],
        'model__max_features': ['sqrt', 'log2']
    }
]

gridsearch_rf = GridSearchCV(rf_pipeline, rf_param_grid, cv=3, scoring="neg_mean_squared_error", n_jobs=-1)
gridsearch_rf.fit(X_train, y_train)

print('Meilleurs paramètres :')
print(gridsearch_rf.best_params_)

rmse_best_rf_model = np.sqrt(-gridsearch_rf.best_score_)
print('RMSE par validation croisée du meilleur modèle : ', rmse_best_rf_model)

best_rf_model = gridsearch_rf.best_estimator_

#### Sauvegarde du meilleur modèle

In [ ]:
dump(best_rf_model, 'california_rf_model.joblib')

#### Analyse de l'importance des variables

Une forêt aléatoire nous permet de savoir quelles sont les variables qui ont le plus d'impact sur la prédiction. Les cellules suivantes permettent d'afficher un graphique de l'importance des variables dans la décision du prédicteur.

In [ ]:
var_names_one_hot = list(best_rf_model.named_steps["preparation"].named_transformers_['cat'].categories_[0])
var_names_one_hot

In [ ]:
var_names = var_names_num + var_names_one_hot
var_names

In [ ]:
def feature_importances(rf_model, var_names, figsize=(12,12)):
    feature_importances = rf_model.feature_importances_
    std_feature_importances = np.std([tree.feature_importances_ for tree in rf_model.estimators_], axis=0)
    print(sorted(zip(feature_importances, std_feature_importances, var_names), reverse=True))
    sorted_indices = np.argsort(feature_importances)[::-1]
    sorted_names = [name for _ , name in sorted(zip(feature_importances, var_names), reverse=True)]
    nb_features = len(feature_importances)
    plt.figure(figsize=figsize)
    plt.title("Importance des variables")
    plt.bar(range(nb_features), feature_importances[sorted_indices],
            color="b", yerr=std_feature_importances[sorted_indices], 
            align="center")
    plt.xticks(range(nb_features), sorted_names, rotation=70)
    plt.xlim([-1, nb_features])
    plt.show()
feature_importances(best_rf_model.named_steps['model'], var_names)

#### Score par validation croisée sur le jeu d'entraînement

In [ ]:
rf_cv_scores = cross_val_score(
    best_rf_model, X_train, y_train, scoring="neg_mean_squared_error", cv=10, n_jobs=-1
)
rf_cv_scores = np.sqrt(-rf_cv_scores)
print(
    "Moyenne des scores : ", rf_cv_scores.mean(),
    "Ecart type des scores : ", rf_cv_scores.std()
)
dump(rf_cv_scores, 'california_rf_score.joblib')

### 7.5.3 Comparaison des modèles

Nous pouvons afficher les scores obtenus par les différents modèles sur les validations croisées effectuées jusqu'ici. Pour choisir un modèle, nous regardons aussi bien les performances moyennes de chaque modèle que l'écart type de leurs performances. Si nous avons plusieurs modèles sensiblement identiques en terme de performance, nous privilegierons celui qui a l'écart type le plus faible pour minimiser le risque d'être confronté à une variance importante lors des tests sur le dataset de test (et lors de l'utilisation en production du modèle retenu).

In [ ]:
figure = plt.figure()
axis = figure.add_subplot(111)
plt.boxplot([ridge_cv_scores, rf_cv_scores])
axis.set_xticklabels(['Ridge', 'Forêt aléatoire'], rotation = 45, ha="right")
axis.set_ylabel("Root Mean Squared Error (RMSE)")
plt.show()

## 7.6 Evaluation du meilleur modèle sur le jeu de test

Finalement nous testons ce modèle sur nos données de test :

In [ ]:
from sklearn.metrics import mean_squared_error

test_pred = best_rf_model.predict(X_test)
test_rmse = np.sqrt(mean_squared_error(y_test, test_pred))
print("Test RMSE : ", test_rmse)

Exemple de réutilisation d'un modèle enregistré (pourrait être dans un script Python séparé ou dans un autre notebook) :

In [ ]:
clf = load('california_rf_model.joblib')
clf.predict(X_test.sample())